In [3]:
!pip install wandb==0.12.1

  Using cached wandb-0.12.1-py2.py3-none-any.whl (1.7 MB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached subprocess32-3.5.4-py3-none-any.whl
  Using cached sentry_sdk-1.30.0-py2.py3-none-any.whl (218 kB)
  Using cached promise-2.3-py3-none-any.whl
  Using cached shortuuid-1.0.11-py3-none-any.whl (10 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalled urllib3-1.26.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.22.38 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.22.38 requires rsa<4.8,>=3.1.2, but you have rsa 4.8 which is incompatible.


In [5]:
!pip uninstall -y torchvision

!pip install torchvision

  Using cached torchvision-0.14.1-cp37-cp37m-manylinux1_x86_64.whl (24.2 MB)


In [2]:
!pip uninstall -y torch

!pip install torch

Found existing installation: torch 1.8.0+cu101
Uninstalling torch-1.8.0+cu101:
  Successfully uninstalled torch-1.8.0+cu101
  Using cached torch-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (887.5 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)


In [21]:
from tqdm import tqdm
from os import listdir
from random import *
seed(0)


import pandas as pd

from pylab import *

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torch.utils.data import DataLoader, random_split

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

import wandb

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cuda:0"

### Data trepanation

In [4]:
dataset = 'barents'
folder_train, folder_val, folder_test = dataset + '/train/maps/', dataset + '/valid/maps/', dataset + '/test/maps/'
train_files, val_files, test_files = sorted([file for file in listdir(folder_train)])[1346:],\
                                     sorted([file for file in listdir(folder_val)]),\
                                     sorted([file for file in listdir(folder_test)]) # + 1200

sample = torch.load(folder_train + train_files[1])
grid = torch.load(dataset + "/train/grid.pt")
coverage = pd.read_csv(dataset + "/train/coverage.csv")

len(train_files), len(val_files), len(test_files)

(1812, 366, 365)

In [5]:
def preprocess_image(tensor: torch.Tensor) -> torch.Tensor:
    return (torch.nan_to_num(tensor, nan=-10.0) + grid['land']*10)

In [6]:
images_train = [preprocess_image(torch.load(folder_train + train_files[i])["jaxa.sic"])\
                            for i in tqdm(range(0, len(train_files)))]

images_val = [preprocess_image(torch.load(folder_val + val_files[i])["jaxa.sic"])\
                            for i in tqdm(range(0, len(val_files)))]

def handler(data: dict):
    try:
        return data["jaxa.sic"]
    except:
        return data["jaxa.sic_asc"]
    
images_test = [preprocess_image(handler(torch.load(folder_test + test_files[i])))\
                            for i in tqdm(range(0, len(test_files)))]

images_tensor_train, images_tensor_val, images_tensor_test = torch.stack(images_train).to(device),\
                                                             torch.stack(images_val).to(device),\
                                                             torch.stack(images_test).to(device)
images_tensor_train.shape

100%|██████████| 365/365 [00:06<00:00, 52.29it/s]


torch.Size([1812, 360, 500])

In [7]:
def average(data: list) -> float:
    return sum(data)/len(data)

def load_data(batch_size, images_tensor, d_in, d_out):
    in_batch, out_batch = [], []
    for i in range(batch_size):
        start_point = choice(range(len(images_tensor) - d_in - d_out))
        in_batch.append(images_tensor[start_point:start_point+d_in])
        out_batch.append(images_tensor[start_point+d_in:start_point+d_in+d_out])
    return torch.stack(in_batch), torch.stack(out_batch)

def train(model, criterion, d_in, d_out, epochs, batch_size, lr, eval_step):
    optimizer = optim.Adam(model.parameters(), lr=lr) #weight_decay=1e-8, momentum=0.9
#    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
#    grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
    for epoch in range(1, epochs+1):
#        print("Current epoch", epoch)
        model.train()
        images_in, images_out = load_data(batch_size, images_tensor_train, d_in, d_out)
        optimizer.zero_grad()
        model_out = model(images_in)
        loss = criterion(model_out, images_out)
        loss.backward(retain_graph=True)
        optimizer.step()
        if epoch % eval_step == 0:
            mae_total, rmse_total, mape_total = [], [], []
            
            total_test_len = len(images_tensor_test)
            start_point, steps = 0, int(total_test_len/(d_in + d_out))
            
            for i in range(steps):
                images_in, images_out = images_tensor_test[start_point:start_point+d_in],\
                                        images_tensor_test[start_point+d_in:start_point+d_in+d_out]
                
                start_point += d_in + d_out
            
                model_out = model(images_in[None, :, :, :])
                
                criterion_mse = nn.MSELoss()
                
                loss_mse = criterion_mse(model_out, images_out[None, :, :, :])
                loss_mae = criterion(model_out, images_out[None, :, :, :])
                
                mae_total.append(loss_mae.detach().cpu().numpy())
                rmse_total.append(math.sqrt(loss_mse.detach().cpu().numpy()))
                
            
            wandb.log(
                {
                 "test/test_MAE": average(mae_total), 
                 "test/test_RMSE": average(rmse_total)
                }
            )
            
            print('Epoch ', epoch, ', test MAE - ', average(mae_total))

In [21]:
architecture = "ResNet_default_7:10"
i = 0
wandb.init(project="SeaIcePrediction", entity="eighonet", group=architecture)
wandb.run.name = architecture + "_" + str(i)  # "gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
wandb.run.save()

wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-07-26 15:05:37.509437: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [7]:
import torchvision.models

In [8]:
from torchvision.models import resnet50

In [9]:
from torchvision import transforms, models

In [10]:
model = models.resnet152(pretrained=False)
#model = resnet50()

/home/jovyan/.imgenv-pensive-herschel-0/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/jovyan/.imgenv-pensive-herschel-0/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [27]:
model.conv1 = nn.Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model = torch.nn.Sequential(*(list(model.children())[:-2])).to(device)

In [28]:
test_input = torch.zeros(8, 7, 360, 500).to(device)

In [29]:
model(test_input).shape

torch.Size([8, 2048, 12, 16])

In [30]:
model

Sequential(
  (0): Conv2d(7, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [31]:
conv_int1 = nn.Conv2d(1024, 1024, kernel_size=3, padding=1, bias=False).to(device)
conv_int2 = nn.Conv2d(512, 511, kernel_size=3, padding=1, bias=False).to(device)

up1 = nn.ConvTranspose2d(2048, 2048 // 2, kernel_size=2, stride=(2,2)).to(device)
up2 = nn.ConvTranspose2d(1024, 1024 // 2, kernel_size=3, stride=(3,3)).to(device)
up3 = nn.ConvTranspose2d(511, 511 // 170, kernel_size=(5, 25), stride=(5,5)).to(device)
#up4 = nn.ConvTranspose2d(3, 3, kernel_size=(1,21), stride=(1,1)).to(device)

#print(resnet152_aug(test_input).shape)
print(conv_int1(up1(model(test_input))).shape)
print(up2(up1(model(test_input))).shape)
print(conv_int2(up2(up1(model(test_input)))).shape)
print(up3(conv_int2(up2(up1(model(test_input))))).shape)
#print(up4(up3(conv_int2(up2(up1(model(test_input)))))).shape)
#print(up5(up4(up3(up2(up1(model(test_input)))))).shape)
#print(up6(up5(up4(up3(up2(up1(model(test_input))))))).shape)

torch.Size([8, 1024, 24, 32])
torch.Size([8, 512, 72, 96])
torch.Size([8, 511, 72, 96])
torch.Size([8, 3, 360, 500])


In [16]:
511 // 10

51

In [32]:
model.requires_grad = True

In [33]:
resnet152_aug = nn.Sequential(
    model,
    nn.ConvTranspose2d(2048, 2048 // 2, kernel_size=2, stride=(2,2)),
    nn.Conv2d(1024, 1024, kernel_size=3, padding=1, bias=False),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(1024, 1024 // 2, kernel_size=3, stride=(3,3)),
    nn.Conv2d(512, 511, kernel_size=3, padding=1, bias=False),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(511, 511 // 170, kernel_size=(5, 25), stride=(5,5)).to(device)
).to(device)

In [1]:
import wandb

In [2]:
architecture = "Per_7:3_per"
i = 0
wandb.init(project="initial_tests_RE", entity="sea-ice-prediction", group=architecture)
wandb.run.name = architecture + "_" + str(i)  # "gs_3_ffd_3_128_test_MAEloss_lr_10^{-4}_10000"
wandb.run.save()

wandb: Currently logged in as: uva-ml (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.15.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [4]:
for i in range(10000):
    wandb.log(
        {
         "test/test_MAE": 0.02706, 
         "test/test_RMSE": 0.08123
        }
    )    

In [33]:
resnet152_aug(test_input).shape

torch.Size([8, 3, 360, 500])

In [34]:
torch.save(resnet152_aug, 'resnet_test.pt')

In [22]:
from models.unet.unet.unet_model import UNet

epochs = 1500
d_in, d_out = 7, 10
batch_size = 8
lr = 1e-3
eval_step = 20
criterion = nn.L1Loss()
model_a = resnet152_aug.to(device)

train(resnet152_aug, criterion, d_in, d_out, epochs, batch_size, lr, eval_step)

Epoch  20 , test MAE -  4.348831903366816
Epoch  40 , test MAE -  3.5120751517159596
Epoch  60 , test MAE -  3.979979015531994
Epoch  80 , test MAE -  3.9091353643508184
Epoch  100 , test MAE -  3.9249165852864585
Epoch  120 , test MAE -  3.8048789614722844
Epoch  140 , test MAE -  3.87554931640625
Epoch  160 , test MAE -  3.8276294526599703
Epoch  180 , test MAE -  3.938957940964472
Epoch  200 , test MAE -  3.891422816685268
Epoch  220 , test MAE -  3.933380126953125
Epoch  240 , test MAE -  4.032685416085379
Epoch  260 , test MAE -  3.9244882492792037
Epoch  280 , test MAE -  3.690108163016183
Epoch  300 , test MAE -  3.9189747401646207
Epoch  320 , test MAE -  4.380914233979725
Epoch  340 , test MAE -  3.5831749325706843
Epoch  360 , test MAE -  3.6536512829008556
Epoch  380 , test MAE -  3.577599298386347
Epoch  400 , test MAE -  3.8338085356212797
Epoch  420 , test MAE -  3.9339595976329984
Epoch  440 , test MAE -  3.799738566080729
Epoch  460 , test MAE -  3.675442650204613
Epoch

In [23]:
train(resnet152_aug, criterion, d_in, d_out, 10000, 16, 1e-4, eval_step)

RuntimeError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 2; 31.72 GiB total capacity; 29.65 GiB already allocated; 26.44 MiB free; 30.38 GiB reserved in total by PyTorch)

In [ ]:
train(resnet152_aug, criterion, d_in, d_out, 10000, 16, 1e-4, eval_step)